In [6]:
import pickle 
import requests 
import json
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [7]:
api_key = 'RGAPI-6ad7996e-b34b-47c3-b226-e4b24ed2744f'
current_version = requests.get('https://ddragon.leagueoflegends.com/api/versions.json').json()[0]
current_version

'10.22.1'

In [26]:
# 챔피언 종류 받아와서 아이디, key 따로 저장하기
champion_dict = {}
r = requests.get('http://ddragon.leagueoflegends.com/cdn/{}/data/ko_KR/champion.json'.format(current_version))
info_df = pd.DataFrame(r.json())
for i, champ in enumerate(info_df.data):
    champion_dict[champ['id']] = champ['key']

In [23]:
# 챌~마스터 구간 유저 4300명 값 불러와서 dataframe으로 만들고 필요없는 컬럼들 제거하기

api_challenger = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
api_grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
api_master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key

league_df = pd.DataFrame(requests.get(api_challenger).json())
r = requests.get(api_grandmaster)
league_df = pd.concat([league_df, pd.DataFrame(r.json())], axis=0)
r = requests.get(api_master)
league_df = pd.concat([league_df, pd.DataFrame(r.json())], axis=0)

league_df.reset_index(inplace=True)
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T
league_df = pd.concat([league_df, league_entries_df], axis=1)
league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank', 'freshBlood', 'hotStreak', 'inactive', 'veteran', 'wins', 'losses'], axis=1)
league_df

,tier,leaguePoints,summonerId,summonerName
0,CHALLENGER,905,QHOBZcnCVQuDhVo0hqDYJDHcKXeV8mJgAXY53aFZyYC5-lc,진철수삼촌
1,CHALLENGER,852,8sRiLvDv1a30gLvXGCf6po3B_pbUqBQEHJOYdaXWZWP593I,HLE Baut
2,CHALLENGER,1190,4OSSiuVysOWwKVM2GMpBQmIj6QDQnSy5mOp8Mth6LqHhTpk,윤용호
3,CHALLENGER,811,bDSuD49dPMGZdxlytAMgp__IVfQpc7xUo7sXxlcerrozs1A,DRX Deft
4,CHALLENGER,849,t6rjOClbuxtRb11NMsDa98h13Am0Vnl37X6Wz10NnA2dPgM,LJL강해질것이다
5,CHALLENGER,859,LVbrTfD-f8_ZK_4m6l8kkEwoGy0nZTqYY2XSgxvsM84cOw,하이드
6,CHALLENGER,847,fB4tbAMqIdCP_jMffnJFEHHJugfJhRErGNM7hZ2PA-x3kAc,순 의
7,CHALLENGER,830,YyC22_lXkkbrdnP6GzhU9PwW-LcJSPuhQ8oS5li4M-c4DUE,ke da ya 1hao
8,CHALLENGER,1003,VDlvpYvs1I5dtYS7SMiox1qCw47GI3gilZUz-3XI5PiieDs,엘 림
9,CHALLENGER,1369,3Spui0kDuXnKJ3-S9SHOBWMsVrN07CV31E7kbnprbIuRNZ8,IGqq1019458971


In [24]:
# summonerID값으로 accountID 불러와서 df에 합치기

league_df['account_id'] = np.nan
for i, summoner_id in enumerate(league_df[:20]['summonerId']):
    api_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/' + summoner_id + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200:
        time.sleep(5)
        r = requests.get(api_url)
    account_id = r.json()['accountId']
    league_df.iloc[i, -1] = account_id

In [25]:
league_df

,tier,leaguePoints,summonerId,summonerName,account_id
0,CHALLENGER,905,QHOBZcnCVQuDhVo0hqDYJDHcKXeV8mJgAXY53aFZyYC5-lc,진철수삼촌,W4kFL9YW_sLrVmyl1pzRq6CvOCgclEAa74eWkCImGazupMA
1,CHALLENGER,852,8sRiLvDv1a30gLvXGCf6po3B_pbUqBQEHJOYdaXWZWP593I,HLE Baut,qOO6KrqO_4Y7c31Fe7Rok1UD5HoODyvkAE9lTZNkydv6trY
2,CHALLENGER,1190,4OSSiuVysOWwKVM2GMpBQmIj6QDQnSy5mOp8Mth6LqHhTpk,윤용호,Pvho23uRuBjzWWV91ToL1fTaWLb_PIH6ViRWdnO9WUSoEdw
3,CHALLENGER,811,bDSuD49dPMGZdxlytAMgp__IVfQpc7xUo7sXxlcerrozs1A,DRX Deft,nnxi2V4iq_9zlOOlAw1vTyKPF79fvLVOli2L-m5Z9Dqs3II
4,CHALLENGER,849,t6rjOClbuxtRb11NMsDa98h13Am0Vnl37X6Wz10NnA2dPgM,LJL강해질것이다,oP-S_e3Y8UHoiUxeuA-LP2f0PPL7drbLYSIj_ZBEOYpZxJg
5,CHALLENGER,859,LVbrTfD-f8_ZK_4m6l8kkEwoGy0nZTqYY2XSgxvsM84cOw,하이드,27RDXq-zwjjPZCYCEEKomjhEkGAkIcG26-DMAs9xUOM2
6,CHALLENGER,847,fB4tbAMqIdCP_jMffnJFEHHJugfJhRErGNM7hZ2PA-x3kAc,순 의,EX5eHrmokssOxH5x3hSrIyjJD_t9jkR-4fxFwFMxHhMB
7,CHALLENGER,830,YyC22_lXkkbrdnP6GzhU9PwW-LcJSPuhQ8oS5li4M-c4DUE,ke da ya 1hao,6gv4wE8ANQp3jm8-Cac3WDPinfm0B4_vf3AXMAm8TPg6Ca...
8,CHALLENGER,1003,VDlvpYvs1I5dtYS7SMiox1qCw47GI3gilZUz-3XI5PiieDs,엘 림,j-1I4pBR1x_O-sxLI9VpBpaO4cWclEkGIyAgLXdPcKwii-0
9,CHALLENGER,1369,3Spui0kDuXnKJ3-S9SHOBWMsVrN07CV31E7kbnprbIuRNZ8,IGqq1019458971,ZZNhuj8YVD2mOiuSDrGm4m0oyYkJEaVHDNohZGIVytetDv...


In [27]:
# accountID로 순서대로 1계정당 100개씩 전적 불러오기 (테스트로 20개)

season = str(13)
match_info_df = pd.DataFrame()
for account_id in league_df[:20]['account_id']:
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + account_id + \
                  '?season=' + season + '&api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200:
        time.sleep(5)
        r = requests.get(api_url)
    match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])

In [28]:
match_info_df

,champion,gameId,lane,platformId,queue,role,season,timestamp
0,81,4770455908,BOTTOM,KR,430,DUO_CARRY,13,1604826932971
1,22,4768075412,TOP,KR,450,DUO_SUPPORT,13,1604747802269
2,875,4765999482,JUNGLE,KR,430,NONE,13,1604673534258
3,157,4766003163,NONE,KR,430,DUO,13,1604672041771
4,104,4765525877,JUNGLE,KR,430,NONE,13,1604660372961
5,875,4764861408,NONE,KR,420,DUO_SUPPORT,13,1604643039471
6,55,4764678168,MID,KR,420,SOLO,13,1604641066918
7,875,4764027351,MID,KR,420,DUO_SUPPORT,13,1604584146410
8,875,4763096670,BOTTOM,KR,420,DUO_SUPPORT,13,1604564130350
9,875,4762448589,BOTTOM,KR,420,DUO_SUPPORT,13,1604504409437


In [29]:
# gameID가 겹치면 삭제. 상위티어이기 때문에 약 5%가량 발생

match_info_df = match_info_df.drop_duplicates('gameId')

In [30]:
match_info_df

,champion,gameId,lane,platformId,queue,role,season,timestamp
0,81,4770455908,BOTTOM,KR,430,DUO_CARRY,13,1604826932971
1,22,4768075412,TOP,KR,450,DUO_SUPPORT,13,1604747802269
2,875,4765999482,JUNGLE,KR,430,NONE,13,1604673534258
3,157,4766003163,NONE,KR,430,DUO,13,1604672041771
4,104,4765525877,JUNGLE,KR,430,NONE,13,1604660372961
5,875,4764861408,NONE,KR,420,DUO_SUPPORT,13,1604643039471
6,55,4764678168,MID,KR,420,SOLO,13,1604641066918
7,875,4764027351,MID,KR,420,DUO_SUPPORT,13,1604584146410
8,875,4763096670,BOTTOM,KR,420,DUO_SUPPORT,13,1604564130350
9,875,4762448589,BOTTOM,KR,420,DUO_SUPPORT,13,1604504409437


In [37]:
# 위에서 얻어낸 게임ID를 가지고 실제 게임데이터를 받아옴

match_df = pd.DataFrame()
for game_id in match_info_df[:20]['gameId']:
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matches/' + str(game_id) + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200:
        time.sleep(5)
        r = requests.get(api_url)
    r_json = r.json()
    temp_df = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T
    match_df = pd.concat([match_df, temp_df])

In [33]:
match_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4770455908,KR,1604826932971,1335,430,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4768075412,KR,1604747802269,1452,450,12,13,10.22.341.643,ARAM,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4765999482,KR,1604673534258,2026,430,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4766003163,KR,1604672041771,1174,430,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4765525877,KR,1604660372961,1942,430,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4764861408,KR,1604643039471,934,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4764678168,KR,1604641066918,1389,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4764027351,KR,1604584146410,1941,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4763096670,KR,1604564130350,1808,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,4762448589,KR,1604504409437,1483,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [43]:
# 게임 내 여러가지 테마의 게임이 있지만, 소환사의협곡 솔로랭크와 팀랭크만 수집

select_indices = (match_df['queueId']==420) | (match_df['queueId']==440)
match_df = match_df.loc[select_indices, :].reset_index(drop=True)
match_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4764861408,KR,1604643039471,934,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,4764678168,KR,1604641066918,1389,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,4764027351,KR,1604584146410,1941,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,4763096670,KR,1604564130350,1808,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,4762448589,KR,1604504409437,1483,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5,4761801940,KR,1604493390765,1899,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
6,4761615755,KR,1604488940430,1566,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
7,4761559669,KR,1604487079603,1314,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
8,4761522003,KR,1604484221104,1715,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
9,4761398378,KR,1604482714381,1038,420,11,13,10.22.341.643,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [40]:
# match_df에 3가지 team, participants, participantIdentities들이 문자열로 받아지는 현상 방지, 1회만 하면 됨

# for column in ['teams', 'participants', 'participantIdentities']:
#     match_df[column] = match_df[column].map(lambda v: eval(v))

TypeError: eval() arg 1 must be a string, bytes or code object

In [42]:
# 테스트용으로 받아온 15경기의 챔피언 번호 받아오기

champion = []
for i in range(len(match_df)):
    champion = []
    for _ in range(10):
        # 탑 0, 5 정글 1, 6 미드 2, 7 원딜 3, 8 서폿 4, 9로 맞추기 위한 작업필요
        # 0~4, 5~9는 이긴 팀이 앞쪽으로 맞추기
        championNum = match_df['participants'][i][_]['championId']
        win = match_df['participants'][i][_]['stats']['win']
        lane = match_df['participants'][i][_]['timeline']['lane']
        champion.append(championNum)
    print(champion)

[164, 84, 412, 64, 21, 145, 78, 875, 104, 23]
[164, 91, 12, 115, 107, 89, 78, 55, 202, 104]
[875, 145, 13, 78, 114, 61, 104, 412, 429, 117]
[4, 164, 163, 421, 555, 517, 64, 58, 875, 498]
[58, 76, 875, 517, 235, 84, 12, 360, 104, 117]
[875, 8, 523, 104, 3, 164, 81, 44, 55, 76]
[235, 875, 106, 120, 777, 69, 360, 58, 427, 80]
[58, 106, 80, 202, 142, 4, 104, 81, 875, 24]
[64, 875, 154, 202, 3, 236, 412, 1, 164, 421]
[154, 107, 85, 360, 875, 58, 30, 91, 89, 235]
[875, 76, 21, 126, 24, 517, 81, 164, 64, 43]
[81, 164, 104, 142, 432, 429, 876, 68, 412, 24]
[245, 202, 39, 89, 126, 5, 80, 875, 236, 142]
[236, 432, 84, 104, 81, 22, 875, 517, 76, 58]
[150, 89, 61, 360, 203, 412, 523, 104, 245, 58]
